# Generative models

In probabilistic discriminative models like logistic regression, we model the posterior distribution $\mathbb{P}\{Y=k|X\}$ directly. In generative models, however, we instead model the joint distribution 

$$\mathbb{P}\{X, Y\}=\mathbb{P}\{X|Y=k\}\mathbb{P}\{Y=k\}$$

In practice, this is done by modeling the likelihood $\mathbb{P}\{X|Y=k\}$ and the prior $\mathbb{P}\{Y=k\}$ separate. Once these distributions have been modeled, we can recontruct the posterior using Bayes theorem

$$\mathbb{P}\{Y=k|X\}=\frac{\mathbb{P}\{X|Y=k\}\mathbb{P}\{Y=k\}}{\sum_{k} \mathbb{P}\{X|Y=k\}\mathbb{P}\{Y=k\}}$$

During inference, given the features $X$, one can compute the posterior and choose the class $k$ that maximizes the posterior distribution. 

$$\hat{k}=\underset{k}{\text{argmin}}\; \mathbb{P}\{Y=k|X\} = \underset{k}{\text{argmin}} \;\mathbb{P}\{X|Y=k\}\mathbb{P}\{Y=k\} \tag{1}$$

Note that since we are only comparing the posterior probability for each class, we can drop the normalizing factor in $(1)$. In this section, we discuss several generative models in machine learning. 